# DiC Assignment 2

Group 6
Members:
 Theresa Mayer
 Theresa Bruckner
 Jan Tölken
 Can Kenan Kandil 
 Thomas Klar


## Part 1

## Part 2

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF, ChiSqSelector, IndexToString, StringIndexer, CountVectorizer
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Assignment2").getOrCreate()

In [3]:
path = "reviews_devset.json" #"hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
input_file = spark.read.format("json").load(path).select("category", "reviewText")

In [4]:
input_file.show(n=5)

+--------------------+--------------------+
|            category|          reviewText|
+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...|
|Patio_Lawn_and_Garde|This is a very ni...|
|Patio_Lawn_and_Garde|The metal base wi...|
|Patio_Lawn_and_Garde|For the most part...|
|Patio_Lawn_and_Garde|This hose is supp...|
|Patio_Lawn_and_Garde|This tool works v...|
|Patio_Lawn_and_Garde|This product is a...|
|Patio_Lawn_and_Garde|I was excited to ...|
|Patio_Lawn_and_Garde|I purchased the L...|
|Patio_Lawn_and_Garde|Never used a manu...|
|Patio_Lawn_and_Garde|Good price. Good ...|
|Patio_Lawn_and_Garde|I have owned the ...|
|Patio_Lawn_and_Garde|I had "won" a sim...|
|Patio_Lawn_and_Garde|The birds ate all...|
|Patio_Lawn_and_Garde|Bought last summe...|
|Patio_Lawn_and_Garde|I knew I had a mo...|
|Patio_Lawn_and_Garde|I was a little wo...|
|Patio_Lawn_and_Garde|I have used this ...|
|Patio_Lawn_and_Garde|I actually do not...|
|Patio_Lawn_and_Garde|Just what 

### Label Encoding

In [5]:
indexer = StringIndexer(inputCol="category", outputCol="label")
indexModel = indexer.fit(input_file)
input_file_1 = indexModel.transform(input_file)

In [6]:
reindexer = IndexToString(inputCol=indexer.getOutputCol(), outputCol="category_reindexed")
reindexer.transform(input_file_1).show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|  category_reindexed|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|For the most part...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|Patio_Lawn_and_Garde|
+--------------------+--------------------+-----+--------------------+


### Tokenization

In [8]:
tokenizer = RegexTokenizer(inputCol='reviewText', outputCol='tokens', pattern=r"[ \t\d(){}\[\].!?;:,\-=\"~#@&*%€$§\\'\n\r\/]+", minTokenLength=2, toLowercase=True)

In [9]:
input_2 = tokenizer.transform(input_file_1)
input_2.show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|              tokens|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|[this, was, gift,...|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|[this, is, very, ...|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|[the, metal, base...|
|Patio_Lawn_and_Garde|For the most part...| 18.0|[for, the, most, ...|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|[this, hose, is, ...|
+--------------------+--------------------+-----+--------------------+


### Stopword Removal

In [10]:
stopword_file = "stopwords.txt"
with open(stopword_file, 'r', encoding='utf-8') as f:
    # Strip whitespace and convert to lowercase
    stopwords = [line.strip() for line in f]

In [11]:
stopword_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                                    outputCol="tokens_nostop",
                                    stopWords=stopwords)

In [12]:
input_3 = stopword_remover.transform(input_2)
input_3.select("tokens", "tokens_nostop").show(n=5)

+--------------------+--------------------+-----+--------------------+--------------------+
|            category|          reviewText|label|              tokens|       tokens_nostop|
+--------------------+--------------------+-----+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|[this, was, gift,...|[gift, husband, m...|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|[this, is, very, ...|[nice, spreader, ...|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|[the, metal, base...|[metal, base, hos...|
|Patio_Lawn_and_Garde|For the most part...| 18.0|[for, the, most, ...|[part, works, pre...|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|[this, hose, is, ...|[hose, supposed, ...|
+--------------------+--------------------+-----+--------------------+--------------------+


### TF-IDF Calculation

In [13]:
tf = CountVectorizer(inputCol=stopword_remover.getOutputCol(), 
                      outputCol="tf_output", 
                      vocabSize=40_000)

In [14]:
idf = IDF(inputCol=tf.getOutputCol(), 
          outputCol="tfidf_output",
          minDocFreq=4)

In [15]:
tfmodel = tf.fit(input_3)
input_4 = tfmodel.transform(input_3)
input_4.select("tokens_nostop", "tf_output").show(n=5)

+--------------------+
|           tf_output|
+--------------------+
|(40000,[2,3,7,8,3...|
|(40000,[0,1,3,21,...|
|(40000,[4,10,29,1...|
|(40000,[1,3,4,9,1...|
|(40000,[12,32,42,...|
+--------------------+


In [16]:
idfModel = idf.fit(input_4)
input_5 = idfModel.transform(input_4)
input_5.select("tf_output", "tfidf_output").show(n=5)

+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|            category|          reviewText|label|              tokens|       tokens_nostop|           tf_output|        tfidf_output|
+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|[this, was, gift,...|[gift, husband, m...|(40000,[2,3,7,8,3...|(40000,[2,3,7,8,3...|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|[this, is, very, ...|[nice, spreader, ...|(40000,[0,1,3,21,...|(40000,[0,1,3,21,...|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|[the, metal, base...|[metal, base, hos...|(40000,[4,10,29,1...|(40000,[4,10,29,1...|
|Patio_Lawn_and_Garde|For the most part...| 18.0|[for, the, most, ...|[part, works, pre...|(40000,[1,3,4,9,1...|(40000,[1,3,4,9,1...|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|[this, hose, 

### Selection of top 2000 features

In [17]:
chisq = ChiSqSelector(featuresCol=idf.getOutputCol(),
                      labelCol="label",
                      outputCol="features",
                      numTopFeatures=2000)

In [18]:
chisqModel = chisq.fit(input_5)
input_6 = chisqModel.transform(input_5)
input_6.select("features").show(n=5)

+--------------------+
|            features|
+--------------------+
|(2000,[2,3,7,8,35...|
|(2000,[0,1,3,21,3...|
|(2000,[4,10,174,3...|
|(2000,[1,3,4,9,10...|
|(2000,[12,29,101,...|
+--------------------+


### Pipeline Creation

In [19]:
def get_pipeline(n_features=2000):
    chisq.setNumTopFeatures(n_features)
    pipeline = Pipeline(stages=[
        indexer,
        tokenizer,
        stopword_remover,
        tf,
        idf,
        chisq
    ])
    return pipeline

In [20]:
pipeline = get_pipeline(n_features=2000)
preprocessing_pipeline = pipeline.fit(input_file)
preprocessing_pipeline.transform(input_file).select("label", "features").show(n=5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 18.0|(2000,[2,3,7,8,35...|
| 18.0|(2000,[0,1,3,21,3...|
| 18.0|(2000,[4,10,174,3...|
| 18.0|(2000,[1,3,4,9,10...|
| 18.0|(2000,[12,29,101,...|
| 18.0|(2000,[0,3,4,8,11...|
| 18.0|(2000,[18,112,175...|
| 18.0|(2000,[6,21,32,36...|
| 18.0|(2000,[3,4,5,6,40...|
| 18.0|(2000,[6,8,38,78,...|
| 18.0|(2000,[1,13,226],...|
| 18.0|(2000,[5,17,33,40...|
| 18.0|(2000,[1,11,28,35...|
| 18.0|(2000,[40,144,339...|
| 18.0|(2000,[0,3,7,9,11...|
| 18.0|(2000,[8,26,57,80...|
| 18.0|(2000,[1,15,120,1...|
| 18.0|(2000,[2,3,221,26...|
| 18.0|(2000,[4,10,16,20...|
| 18.0|(2000,[0,18,30,42...|
+-----+--------------------+


### Export most important tokens to file

In [21]:
def get_top_terms_from_pipeline(pipeline):
    n = len(pipeline.stages[5].selectedFeatures)


    vocab = pipeline.stages[3].vocabulary.copy()
    top_words = " ".join(sorted([vocab[i] for i in pipeline.stages[5].selectedFeatures]))
    
    with open("output_ds.txt", "w") as f:
        f.write(top_words)
        
    return n

In [22]:
get_top_terms_from_pipeline(preprocessing_pipeline)

2000

# Part 3

To develop the SVM you can use this dataframe for testing

In [23]:
input_test = preprocessing_pipeline.transform(input_file).select("label", "features")
input_test.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 18.0|(2000,[2,3,7,8,35...|
| 18.0|(2000,[0,1,3,21,3...|
| 18.0|(2000,[4,10,174,3...|
| 18.0|(2000,[1,3,4,9,10...|
| 18.0|(2000,[12,29,101,...|
| 18.0|(2000,[0,3,4,8,11...|
| 18.0|(2000,[18,112,175...|
| 18.0|(2000,[6,21,32,36...|
| 18.0|(2000,[3,4,5,6,40...|
| 18.0|(2000,[6,8,38,78,...|
| 18.0|(2000,[1,13,226],...|
| 18.0|(2000,[5,17,33,40...|
| 18.0|(2000,[1,11,28,35...|
| 18.0|(2000,[40,144,339...|
| 18.0|(2000,[0,3,7,9,11...|
| 18.0|(2000,[8,26,57,80...|
| 18.0|(2000,[1,15,120,1...|
| 18.0|(2000,[2,3,221,26...|
| 18.0|(2000,[4,10,16,20...|
| 18.0|(2000,[0,18,30,42...|
+-----+--------------------+


If you are finished and have a working SVM, you can create an end to end pipeline like this: 

In [ ]:
def get_model_pipeline(n_features=2000, add more arguments here):
    chisq.setNumTopFeatures(n_features)
    # set model parameters here or something probably
    model = Pipeline(stages=[
        indexer,
        tokenizer,
        stopword_remover,
        tf,
        idf,
        chisq,
        # add all new transformers/estimators here 
    ])
    return model

Then you can do end-to-end testing with train and test set similar to this:

testset, trainset = split(input_data)
model_pipeline = get_model_pipeline(parameters)
model = model_pipeline.fit(trainset)
output = model.transform(testset)
calculate_metrics(output)